In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark import SparkContext

In [4]:
findspark.init()

In [5]:
sc = SparkContext.getOrCreate() # default spark context 생성

In [17]:
sc.stop() 
# sparkContext를 여러개 만들면 안돌아간다. 그래서 새로운 context생성시 먼저 stop을 해줌.

In [18]:
sc = SparkContext(master = 'local[2]', appName = 'Spark Test')

In [7]:
rdd  = sc.parallelize([1,2,3])

In [8]:
rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:184

In [11]:
# rdd 는 가상의 리스트다. 무엇인가를 가지고있다는 정보만 들고있다.

In [16]:
rdd.filter(lambda x: x >1).count()

2

filter.map.map 이렇게 계속하다가 실제 연산을 원하면 count, action 을 함

map,filter 많이쓰이는 transformation 

In [17]:
rdd.collect() # rdd 를 뽑아보면, 123,으로만들었으니까 1,2,3 그대로있다.

[1, 2, 3]

In [18]:
rdd.map(lambda x:x*2).collect()

[2, 4, 6]

In [19]:
rdd.collect()

[1, 2, 3]

In [ ]:
# list 의 list 를 하나의 list 로 줄이는것.

In [33]:
llist = [[1,2],[3,4]]

In [34]:
rdd2 =sc.parallelize(llist)

In [35]:
rdd2.collect()

[[1, 2], [3, 4]]

In [38]:
#이것을 하나의 리스트로 
rdd2.flatMap(lambda x:(x,x)).collect()

[[1, 2], [1, 2], [3, 4], [3, 4]]

In [39]:
rdd2.collect()

[[1, 2], [3, 4]]

In [41]:
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize([2,3,4])

In [42]:
rdd1.intersection(rdd2)# 트랜스포메이션이기때문에 계산도안하고, 시간도안걸림 따라서 collect를 아래서 달아줌

PythonRDD[27] at RDD at PythonRDD.scala:49

In [43]:
rdd1.intersection(rdd2).collect()

[2, 3]

In [ ]:
#collect 는 데이터가 아주 작을때, 사용해야함, collect 는 rdd 의 결과를 리스트로 반환.
#take(n) 도 많이쓰임 데이터 조회용.
# reduce 는 .

reduce 는 데이터를 합침

In [52]:
rdd1 = sc.parallelize(["1","2","3","4","5"])

In [57]:
rdd1.reduce(lambda x,y: x+y)

'12345'

In [58]:
rdd1.collect()

['1', '2', '3', '4', '5']

In [ ]:
# 하지만 리듀스는,  순서를 보장하지않음.
# 매핑과 리듀스 두가지만으로도 데이터 분석을 많이 할 수 있다.
#reduce 는 액션이다, reducebykey는 데이터가 키랑 벨류로 되어있을때 reduce 하는 방법이다. 리듀스 바이 키는 트랜스포메이션이다.

In [8]:
#rdd = sc.parallelize([1,2,3])

-------------

In [9]:
#rdd.collect()

In [70]:
sc.stop()

In [10]:
from pyspark import SparkConf

In [15]:
conf = SparkConf()

In [16]:
conf.setMaster("local[2]").setAppName("Conf")

In [17]:
sc = SparkContext(conf=conf)

In [68]:
rdd = sc.parallelize([1,2,3,4,5]) # rdd 생성

In [19]:
rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:184

In [69]:
rdd.collect() # 기본적인 rdd를 생성한 것이다.

[1, 2, 3, 4, 5]

In [22]:
rdd.glom().collect() # partition단위로 작업을 한 것이다.

[[1, 2], [3, 4, 5]]

In [23]:
len(rdd.glom().collect()) 
# partition을 2개로 만들어버림. 나중에 파일 2개로 반환해줄것이다.

2

In [72]:
rdd = sc.parallelize([1,2,3,4,5], 3) # partition을 3개로 나눠서 rdd 객체 만들어줌

In [25]:
text = sc.parallelize("./sample/text-test.txt")

In [26]:
text.glom().collect() # partition을 나눠서 한 글자씩 받아옴.
#glom은 각 partition마다 element들을 나눠서 리턴해준다. = transformation

[['.', '/', 's', 'a', 'm', 'p', 'l', 'e', '/', 't', 'e'],
 ['x', 't', '-', 't', 'e', 's', 't', '.', 't', 'x', 't']]

In [27]:
text.collect() # 한 글자씩 다 받아옴.
#collect는 모든 element들을 다 받아오는 것이다. = action

['.',
 '/',
 's',
 'a',
 'm',
 'p',
 'l',
 'e',
 '/',
 't',
 'e',
 'x',
 't',
 '-',
 't',
 'e',
 's',
 't',
 '.',
 't',
 'x',
 't']

In [30]:
rdd = sc.parallelize(["b", "a", "c"], 3) 
# partition 3개로 나눠서 해당 리스트 rdd 객체 생성

In [31]:
rddMap = rdd.map(lambda x:(x,1)) # mapping해줌.
# 특정 형태( key, value 쌍)로 출력해줌.

In [34]:
sorted(rddMap.glom().collect()) 
# partition 3개로 나뉘어져 있는 mapping결과를 출력해줌.

[[('a', 1)], [('b', 1)], [('c', 1)]]

In [35]:
rdd = sc.parallelize([1,2,3,4,5])
rdd.filter(lambda x:x%2==0).collect() # rdd객체에서 filtering한 객체를 출력해줌.

[2, 4]

In [36]:
# flatMap의 경우 각 RDD에 적용시 partition단위로 출력하며, 없는 데이터는 삭제해준다.

rdd = sc.parallelize([2,3,4])
rdd = rdd.map(lambda x:range(1, x))

In [37]:
rdd.glom().collect()

[[range(1, 2)], [range(1, 3), range(1, 4)]]

In [39]:
rdd = sc.parallelize([2,3,4])
rdd.flatMap(lambda x:range(1,x)).collect()

# flatMap의 경우 Mapping해준뒤 1열로 하나의 리스트에 한개씩 쭉 나열해준다.

[1, 1, 2, 1, 2, 3]

In [40]:
rdd = sc.parallelize(["Roses are red", "Violets are blue"])

In [41]:
rdd.map(lambda x:x.split()).collect()

[['Roses', 'are', 'red'], ['Violets', 'are', 'blue']]

In [42]:
rdd = sc.parallelize(["Roses are red", "Violets are blue"])
rdd.flatMap(lambda x:x.split()).collect()
# flatMap의 경우 Mapping해준뒤 1열로 하나의 리스트에 한개씩 쭉 나열해준다.

['Roses', 'are', 'red', 'Violets', 'are', 'blue']

In [43]:
sc.parallelize([1,1,2,3]).distinct().collect()
#distinct()는 중복된 결과를 없애준다.

[2, 1, 3]

In [18]:
#sample은 일부 샘플만 추출해서 가져온다. RDD의 subset을 추출해줌.

rdd = sc.parallelize(range(100), 4)

In [19]:
subset = rdd.sample(False, 0.1, 12)
# 중복 False, 10% 비율, seed =12로 주고 랜덤으로 샘플링 rdd 객체 생성
# 다만 비율이 정확한 값이 아니라 대략 10%를 갖고 오는 것이다.

In [20]:
subset

PythonRDD[19] at RDD at PythonRDD.scala:49

In [21]:
subset.glom().collect()

[[8, 10, 15, 16, 19], [26, 32], [62, 63, 65, 72], [98]]

In [22]:
subset.count() # rdd 객체 내부의 element 개수
# count()의 경우 spark내부 함수. 그래서 len()보다는 더 최적화되어 빠르다.

12

In [23]:
# JOIN
x = sc.parallelize([("a",1), ("b", 4)])
y = sc.parallelize([("a",2)])

In [24]:
# Left Outer Join
sorted(x.leftOuterJoin(y).collect())

[('a', (1, 2)), ('b', (4, None))]

In [25]:
sorted(y.leftOuterJoin(x).collect())

[('a', (2, 1))]

In [57]:
# Right Outer Join
sorted(x.rightOuterJoin(y).collect())

[('a', (1, 2))]

In [58]:
# Join : 키 값이 같은 것들에 대해 join
sorted(x.join(y).collect())

[('a', (1, 2))]

In [59]:
# Intersection : 교집합해줌.
sorted(x.intersection(y).collect()) # 완전히 교집합 되는 것이 없어서 출력되는것이 없음.

[]

In [62]:
# reapartition 
#: 파티션 개수가 다를 경우 join이나 zip할 때 오류가 발생할 수 있어 
# 그것을 방지하기 위해 다시 partition 개수를 수정해준다. 

rdd.glom().count() # 현재 4개의 partition을 가지고 있다.

4

In [63]:
rdd.repartition(10).glom().collect() #

[[],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84],
 [20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94],
 [35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  70,
  71,
  72,
  73,
  74,
  95,
  96,
  97,
  98,
  99],
 [45, 46, 47, 48, 49],
 [],
 [],
 [],
 []]

In [64]:
# http://localhost:4040/ 으로 들어갈 경우 spark에서 작업을 어떻게 수행했는지 볼수 있다.

In [77]:
# take : 하나의 partition에서 상위 몇가지의 element 리턴 = transformation
# cache: 따로 작업공간을 만들어줌. 차후에 꼭!! cache공간을 없애주어야함.
# 그렇지 않는다면 프로세스 종료후에도 남아있는 경우가 있다.
#takeSample : 특정샘플 몇개만 갖고 오는 것. = action

rdd.takeSample(False, 10, 123) # action 함수이기 때문에 따로 print해줄 필요가 없음.

[26, 14, 70, 99, 60, 78, 80, 13, 33, 43]

In [78]:
# reduce : key, value 쌍으로 된 것들을 partition 단위로 reduce 시행.

rdd = sc.parallelize([1,2,3,4,5],2)

In [80]:
rdd.glom().reduce(lambda x,y:x+y)

[1, 2, 3, 4, 5]

In [83]:
#partition 개수에 따라서 생각하는 operation이 아닌 다르게 나올 수 있기 때문에 조심할것.
rdd = sc.parallelize([1,2,.5, .1, 5,.2], 3)
rdd.reduce(lambda x,y:x/y)

# 0.5   ,  5 , 25
# 0.01  , 25
# 0.004

0.004

In [84]:
rdd.glom().collect()

[[1, 2], [0.5, 0.1], [5, 0.2]]

In [9]:
rdd = sc.parallelize([("a",1), ("b",2), ("a",3)])

In [10]:
#reduceByKey : 키 값을 기반으로 value값을 더해서 보여줌.
from operator import add
sorted(rdd.reduceByKey(add).collect())

[('a', 4), ('b', 2)]

In [12]:
# foreach : RDD 객체 내의 element들을 한개 씩 python형태로 넘겨주는데 쓰임.
# foreachPartition : RDD객체의 partition을 한개 씩 python형태로 넘겨주는데 쓰인다.
# saveAsTextFile : RDD객체를 텍스트파일로 저장.

rdd = sc.parallelize([("a",1), ("b",2), ("a",3)], 2)
rdd.saveAsTextFile("abc2")
#위에서 partition을 2개로 나눴기 때문에 2개의 파일에 나뉘어져서 rdd 객체가 저장된다.

In [13]:
fromText = sc.textFile("abc2") #해당 디렉토리 주소를 인자로 주면 해당파일 불러올 수 있다.
fromText.collect()

["('a', 1)", "('b', 2)", "('a', 3)"]

In [14]:
data = sc.parallelize(
    [("a", 22), {"b": 23}, ["c", 4]]
)

In [15]:
obj = data.collect()

In [16]:
type(obj)

list

In [17]:
obj[1]["b"]

23

RDD는 JVM에서 메모리를 잡고 , python에서는 그냥 통신해서 갖고 오는 것임. 

DataFrame은 데이터 타입 정의된 테이블 형태로 반환(익숙한 형태의 스키마)해주기 때문에,<br> row, column에 따라 갖고 올 수 있도록 해준다.

DataFrame 또한 immutable. 수정할 수 없다

DataFrame의 경우 SparkSQL을 쓸 수 있도록 해준다.

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.getOrCreate()

In [8]:
spark

In [8]:
sc = SparkContext.getOrCreate()

In [9]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

위의 spark 세션을 만들던 sc context를 만들던 어짜피 쓰는 드라이버는 똑같다.<br>
껍데기만 다를 뿐이다.

--------------

--------------8.6일---

In [9]:
import findspark

In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.getOrCreate()

In [12]:
sc = SparkContext.getOrCreate()

In [ ]:
# 두개가 같다.


RDD 와 데이터프레임의 차이는 스키마가 있고 없고, 데이터프레임은 raw 데이터를 스키마가 있게 만들어줌. 

In [36]:
df = spark.read.csv("sample/ages.csv")

In [37]:
df.dtypes

[('_c0', 'string'), ('_c1', 'string')]

RDD 였으면 그냥 들어간데로나온다.

read.csv, read.format( 포맷 명시 )

In [38]:
df = spark.read.format('json').load("sample/people.json")

In [39]:
df.dtypes # 컬럼명이랑 데이터타입을 추론함.

[('age', 'bigint'), ('name', 'string')]

In [ ]:
#굉장희 명확하게 구조화된 데이터를 불러온다. jdbc 커넥터로 붙일수있다.

In [40]:
rdd = sc.textFile("sample/people.json") # rdd를 가져온다

In [41]:
rdd

sample/people.json MapPartitionsRDD[54] at textFile at NativeMethodAccessorImpl.java:0

In [42]:
rdd.collect() #똑같은파일인데, rdd 로 가져오면 테이블형식이아님.  데이터프레임으로 되면 : 스키마구조를 갖고 , 자동으로 데이터타입을 지정함

['{"name":"Michael"}',
 '{"name":"Andy", "age":30}',
 '{"name":"Justin", "age":19}']

In [43]:
df = spark.read.json(rdd) #rdd 로 부터 dataframe 을 만든다.  

In [44]:
df

DataFrame[age: bigint, name: string]

# rdd 로 부터 데이터프레임을 만들고 sql 문쓰기 .

In [46]:
findspark.init()

In [47]:
sc

<SparkContext master=local[2] appName=Spark Test>

In [48]:
spark

In [18]:
rdd = sc.parallelize((
"""
    { "id":123,
    "name":"Katy",
    "age":19,
    "eyeColor":"brown"
    }
""",
"""
    {
    "id":124,
    "name":"Joe",
    "age":44,
    "eyeColor":"black"

    }
""",
"""
    {
    "id":125,
    "name":"Romanson",
    "age":25,
    "eyeColor":"blue"

    }

"""
))

In [19]:
rdd.collect() #문자열 그대로 들어감 .처리하려면 regex로 다짤라줘야함 하지만 json 이라는 파일형식을 가지니까. 

['\n    { "id":123,\n    "name":"Katy",\n    "age":19,\n    "eyeColor":"brown"\n    }\n',
 '\n    {\n    "id":124,\n    "name":"Joe",\n    "age":44,\n    "eyeColor":"black"\n\n    }\n',
 '\n    {\n    "id":125,\n    "name":"Romanson",\n    "age":25,\n    "eyeColor":"blue"\n\n    }\n\n']

In [20]:
df = spark.read.json(rdd) #rdd 에 있는 json 을 그대로가져옴 , 컬럼 , 데이터타입 추론해서 rdd 로 부터 dataframe 을 만든다.  

In [21]:
df # 추론된 결과가 나옴

DataFrame[age: bigint, eyeColor: string, id: bigint, name: string]

In [22]:
# pandas 에서 쓰던 dataframe 과는 달리, 여기의 dataframe 은 row 단위로 관리한다 , pandas 의 dataframe은 컬럼으로 관리함.

In [23]:
df.collect()

[Row(age=19, eyeColor='brown', id=123, name='Katy'),
 Row(age=44, eyeColor='black', id=124, name='Joe'),
 Row(age=25, eyeColor='blue', id=125, name='Romanson')]

In [24]:
#실제 테이블 형태로 가져옴. 
df.createOrReplaceTempView("test") #view테이블 만들껀데 view table 의 이름을 test 로 지정 dataframe - > temp view 만듬

df에서 collect 같은 기능이 show 다 .

In [25]:
df.show() # 스키마를 가지기때문에 스키마 형태로 데이터를 보여준다. 

+---+--------+---+--------+
|age|eyeColor| id|    name|
+---+--------+---+--------+
| 19|   brown|123|    Katy|
| 44|   black|124|     Joe|
| 25|    blue|125|Romanson|
+---+--------+---+--------+



In [26]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)



In [27]:
spark.sql("select * from test") #sql 문쓰기 테이블명 만들어줬음 이미 .

DataFrame[age: bigint, eyeColor: string, id: bigint, name: string]

In [28]:
spark.sql("select * from test").collect()

[Row(age=19, eyeColor='brown', id=123, name='Katy'),
 Row(age=44, eyeColor='black', id=124, name='Joe'),
 Row(age=25, eyeColor='blue', id=125, name='Romanson')]

RDD 에서 있던 collect는 스키마없이 컬렉트 반면에, 여기서는 collect가  row 형태로 . 스키마 있는 형태로 리턴.

In [62]:
#새로 RDD 만든다

In [29]:
rdd = sc.parallelize(
    [
        (123,"katie",19,"brown"),
        (124,"joe",45,"black"),
        (125,"romanson",25,"blue")
    ]
)



In [30]:
rdd.collect()

[(123, 'katie', 19, 'brown'),
 (124, 'joe', 45, 'black'),
 (125, 'romanson', 25, 'blue')]

원래 그냥 읽으면 컬럼명 같은것을 다 정해줌 , 하지만 여기서는 우리가 정의할 것.

In [31]:
from pyspark.sql.types import *  # 데이터 타입 다 불러옴

In [32]:
#name, dataType,nullable,metadata 우리가 정해줄거임.

scheme = StructType(
    [
        StructField("id",LongType(),nullable=True),
        StructField("name",StringType(),nullable=True),
         StructField("age",LongType(),nullable=True),
         StructField("eyeColor",StringType(),nullable=True)
    
    ]
)

schema 와 rdd 를 합쳐서 -> dataframe 만들기

In [33]:
df =  spark.createDataFrame(rdd,schema=scheme)

In [34]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



In [35]:
df.createOrReplaceTempView("test2")

In [78]:
# 이제부터는 sql 작업가능해짐
spark.sql("select * from test2").collect()

[Row(id=123, name='katie', age=19, eyeColor='brown'),
 Row(id=124, name='joe', age=45, eyeColor='black'),
 Row(id=125, name='romanson', age=25, eyeColor='blue')]

In [36]:
spark.sql("select count(*)from test").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



#### 세가지 방식

In [37]:
spark.sql("select id,age from test").show() # sql 문 만쓴거.

+---+---+
| id|age|
+---+---+
|123| 19|
|124| 44|
|125| 25|
+---+---+



In [39]:
df.select("id",'age').filter("age=25").show() # sql,함수형 혼합해서 씀

+---+---+
| id|age|
+---+---+
|125| 25|
+---+---+



In [41]:
#데이터프레임 ,카탈리스트 옵티마이저로 최적의경로로 데이터를 불러올것이기때문에 어떤식으로 구성하든 상관이없다.

In [42]:
df.select(df.id,df.age).filter(df.age== 25 ).show() #완전히 함수형으로할경우, filter 함수씀. 대신 python 문법처럼 써야함. 

+---+---+
| id|age|
+---+---+
|125| 25|
+---+---+



In [43]:
flightPath = "sample/departuredelays.csv"
airportPath = "sample/airport-codes-na.txt"

In [49]:
flight = spark.read.csv(flightPath,header=True)

In [50]:
flight.take(1)

[Row(date='01011245', delay='6', distance='602', origin='ABE', destination='ATL')]

In [52]:
airport = spark.read.csv(airportPath,header=True,inferSchema=True,sep="\t")

In [54]:
airport.take(1)

[Row(City='Abbotsford', State='BC', Country='Canada', IATA='YXX')]

In [56]:
#공항정보 , 비행정보 값을 join 하려함.
airport.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- IATA: string (nullable = true)



In [58]:
# sql 쿼리쓰기위해 tempView 를 만들어준다. table 명이 없으니까
flight.createOrReplaceTempView("flight")


In [59]:
airport.createOrReplaceTempView("airport")

In [61]:
spark.sql( # 두 테이블 join 한다. 도시별 delay 를 보려고
    """
        select a.city, f.origin,sum(f.delay) as delay from flight f join airport a on a.IATA = f.origin
        where a.State = "WA"
        group by a.City,f.origin
        order by delay desc

    """
).show()

+-------+------+--------+
|   city|origin|   delay|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



In [65]:
from pyspark.sql import functions as F # sql 의 내부함수

In [ ]:
airport.orderBy()

In [67]:
# 함수형태로 쓰면
airport.join(flight ,airport.IATA==flight.origin)\
        .where(airport.State == "WA")\
        .select(airport.City,flight.origin,sum(flight.delay))\
        .groupby(airport.City,flight.origin)\
        .agg(F.sum(flight.delay))\
        .orderBy("sum(delay)",ascending=False)\
        .show()


TypeError: Column is not iterable

In [68]:
airport.join(flight, airport.IATA == flight.origin)\
.where(airport.State == "WA")\
.select(airport.City, flight.origin, flight.delay)\
.groupby(airport.City, flight.origin)\
.agg(F.sum(flight.delay))\
.orderBy("sum(delay)", ascending=False)\
.show()
 # 오름차순 False -> 내림차순 = 큰값부터 나옴

+-------+------+----------+
|   City|origin|sum(delay)|
+-------+------+----------+
|Seattle|   SEA|  159086.0|
|Spokane|   GEG|   12404.0|
|  Pasco|   PSC|     949.0|
+-------+------+----------+



In [69]:
#우리는 로컬에서 작업을 해주지만 , 실질적으로 카탈리스트가 최적의경로로 자료를 뽑아옴.

# SPARK STREAMING

In [14]:
from pyspark.streaming import StreamingContext

In [21]:
streaming = StreamingContext(sc ,5 ) # 5초마다 스트리밍을 받아온다

In [22]:
streaming #스트리밍 객체

In [8]:
# 어디서 부터 스트리밍을 받아올것인지 소켓을 정해준다 . 받아온 텍스트를 라인이라고한다.
lines = streaming.socketTextStream("localhost",9999)

In [9]:
type(lines) # 이게 RDD 형태이다

pyspark.streaming.dstream.DStream

In [10]:
words = lines.flatMap(lambda line:line.split())

In [11]:
pairs = words.map(lambda w:(w,1))

In [12]:
count = pairs.reduceByKey(lambda a,b : a+b) #키값에 따라 벨류를 더해줌.

In [13]:
count.pprint()

In [17]:
# 스트리밍 받는다고 가정하고 눈에 안보이는것을 디자인하고있는과정.

In [18]:
# streaming 언제 끝날것인가. awaitTermination.

# nc -lvp 9999

In [ ]:
#netcat 은 서버에다가 request  보내고 ,response 받는 용도. 인터넷상에 cat 을 슬수있음.
streaming.start()
streaming.awaitTermination()

-------------------------------------------
Time: 2018-08-06 14:17:00
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:17:05
-------------------------------------------
('apple', 2)

-------------------------------------------
Time: 2018-08-06 14:17:10
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:17:15
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:17:20
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:17:25
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:17:30
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:17:35
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:17:

-------------------------------------------
Time: 2018-08-06 14:22:50
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:22:55
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:23:00
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:23:05
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:23:10
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:23:15
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:23:20
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:23:25
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:23:30
----------

In [27]:
#데이터 프레임을 이용해서 구조적으로 이용한다
lines = spark.readStream.format("socket")\
.option("host","localhost")\
.option("port",9999)\
.load()

In [13]:
from pyspark.sql.functions import explode,split

In [29]:
type(lines)

pyspark.sql.dataframe.DataFrame

In [30]:
words = lines.select(explode(split(lines.value," ")).alias("word")) # word라는 한컬럼을 가지고있는 

In [35]:
type(words)

pyspark.sql.dataframe.DataFrame

In [33]:
wordcount  =  words.groupBy("word").count()

In [ ]:
stream = wordcount.writeStream.outputMode("complete")\
.format("console")\
.start()

stream.awaitTermination() 

---------

# DATA CLEANING 의 부분

In [15]:
from pyspark.sql import SparkSession

In [16]:
spark = SparkSession.builder.getOrCreate()

In [64]:
df = spark.createDataFrame(
    [
        (1, 144.5, 5.9, 33, "M"),
        (2, 167.2, 5.4, 45, "M"),
        (3, 124.1, 5.2, 23, "F"),
        (4, 144.5, 5.9, 33, "M"),
        (5, 133.2, 5.7, 54, "F"),
        (3, 124.1, 5.2, 23, "F"),
        (5, 129.2, 5.3, 42, "M"),    
    ],
    ["id", "weight", "height", "age", "gender"]
)

In [65]:
df.count(), df.distinct().count()

(7, 6)

In [66]:
df = df.dropDuplicates()
df.count()

6

In [67]:
df.orderBy("id").show() #id 같은거는?

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [68]:
df.dropDuplicates([c for c in df.columns if c != "id"]).distinct().count() 
# id만 잘못되었지만, 데이터는 다른디 날리기가 좀 그르네ㅔ....


5

In [69]:
df = df.dropDuplicates([c for c in df.columns if c != "id"])
# 값이 중복된것을 삭제 

In [74]:
df.orderBy("id").show() 

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [75]:
df.count()

5

In [77]:
df.dropDuplicates([c for c in df.columns if c == "id"]).distinct().show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  3| 124.1|   5.2| 23|     F|
|  2| 167.2|   5.4| 45|     M|
+---+------+------+---+------+



In [78]:
from pyspark.sql import functions as f

In [54]:
df.agg(
    f.count("id").alias("Count"),
    f.countDistinct("id").alias("Dcount")  # 중복 제거
).show()

+-----+------+
|Count|Dcount|
+-----+------+
|    7|     5|
+-----+------+



In [55]:
df.withColumn("new_id", f.monotonically_increasing_id()).show() # 자동으로 아이디가 겹치지 않게 생성

+---+------+------+---+------+-------------+
| id|weight|height|age|gender|       new_id|
+---+------+------+---+------+-------------+
|  1| 144.5|   5.9| 33|     M| 171798691840|
|  5| 129.5|   5.2| 44|     M| 601295421440|
|  4| 133.5|   5.9| 33|     M| 670014898176|
|  3| 144.5|   5.2| 23|     F| 996432412672|
|  5| 129.5|   5.2| 42|     M|1108101562368|
|  3| 124.5|   5.7| 23|     F|1503238553600|
|  2| 167.5|   5.4| 45|     M|1709396983808|
+---+------+------+---+------+-------------+



In [56]:
df = df.withColumn("old_id", df.id)

In [ ]:
df = df.withColumn("id", f.monotonically_increasing_id()) 

In [ ]:
df.orderBy("old_id").show()

# cleaning data 에서 계속... 영준씨 필기 참고 


### 판다스 안쓰고 이거 쓰는이유는 이것은 실시간 처리가 가능하다는것!


In [90]:
df = spark.createDataFrame(
    [
        (1, 143.5, 5.6, 28, "M",100000),
        (2, 167.2, 5.4, 45, "M",None),
        (3, None, 5.2,None,None,None),
        (4, 144.5, 5.9, 33, "M",None),
        (5, 133.2, 5.7, 54, "F",None),
        (6, 124.1, 5.2, None, "F",None),
        (7, 129.2, 5.3, 42, "M",76000),    
    ],
    ["id", "weight", "height", "age", "gender","salary"]
)

In [84]:
df.rdd # dataframe 안에 있는데 rdd 다

MapPartitionsRDD[507] at javaToPython at NativeMethodAccessorImpl.java:0

In [85]:
type(df.rdd) # data frame은 안에 rdd 를 가진다

pyspark.rdd.RDD

In [86]:
#원하는 식 뽑아냄

df.rdd.map(
    lambda row:(row["id"], sum([c == None for c in row]))
).collect()

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

In [87]:
# row : 로우별 미싱 벨류 갯수

In [89]:
df.where("id=3").show() # 3번을 보자 . 미싱벨류가 많아서 문제다

+---+------+------+----+------+----+
| id|weight|height| age|gender|  _6|
+---+------+------+----+------+----+
|  3|  null|   5.2|null|  null|null|
+---+------+------+----+------+----+



In [97]:
df.agg(
    *[1 - f.count(c)/f.count("*") for c in df.columns]
      
      ).show() # 미싱벨류 갯수. * : 해당컬럼  비율을 통해서 미싱벨류 찾는다 . 1 이아니니까 미싱벨류가 있다. 14프로가 ㅂ미싱밸류.

+----------------------------+--------------------------------+--------------------------------+-----------------------------+--------------------------------+--------------------------------+
|(1 - (count(id) / count(1)))|(1 - (count(weight) / count(1)))|(1 - (count(height) / count(1)))|(1 - (count(age) / count(1)))|(1 - (count(gender) / count(1)))|(1 - (count(salary) / count(1)))|
+----------------------------+--------------------------------+--------------------------------+-----------------------------+--------------------------------+--------------------------------+
|                         0.0|              0.1428571428571429|                             0.0|           0.2857142857142857|              0.1428571428571429|              0.7142857142857143|
+----------------------------+--------------------------------+--------------------------------+-----------------------------+--------------------------------+--------------------------------+



* [] 이렇게 쓰면. 된다내

In [98]:
df.agg(
    *[(1 - f.count(c)/f.count("*")).alias(c+"Rate") for c in df.columns]
      
      ).show() # 미싱벨류 갯수. * : 해당컬럼  비율을 통해서 미싱벨류 찾는다 . 1 이아니니까 미싱벨류가 있다. 14프로가 미싱밸류. 

+------+------------------+----------+------------------+------------------+------------------+
|idRate|        weightRate|heightRate|           ageRate|        genderRate|        salaryRate|
+------+------------------+----------+------------------+------------------+------------------+
|   0.0|0.1428571428571429|       0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+------+------------------+----------+------------------+------------------+------------------+



In [99]:
# 높을수록 많이 비어있는것이다.  salary는 미싱벨류가 너무많다 -  > 데이터로서의 가치가없다 71 프로나 없다.

In [101]:
df = df.select([c for c in df.columns if c!="salary"]) # salary 컬럼을 뺀 데이터

In [102]:
df.show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  3|  null|   5.2|null|  null|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



In [103]:
df.dropna(thresh = 3 ).show() # null value 가 특정값 이상인 애들을 없엔다. null 값이 몇개나 들어가잇는지 보고. thresh = 3 이면 . 널값 3개이상을 날림.

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



In [112]:
# 모르는 값에 대해서 평균치를 넣을 것이다.
means = df.agg(
    *[f.mean(c).alias(c) for c in df.columns if c!="gender"]  #gender 은 카테고리라서 평균을 하지않는다. null값 뺴고 평균을 계산한다.
)

In [117]:
means.show()

+---+------------------+-----------------+----+
| id|            weight|           height| age|
+---+------------------+-----------------+----+
|4.0|140.28333333333333|5.471428571428572|40.4|
+---+------------------+-----------------+----+



In [113]:
# 여기서 구한 평균값을 빈칸에 넣어주면 평균에 영향을 미치지않는다. 데이터가 크게 영향을 못한다
# 아에 삭제를 하지않는것은 다른 컬럼에 중요한 값이 있을 수 도있어서 예를들먼  height  같은거 때문에 없에지는 않는다.

In [114]:
#이제 수정하자 , 하지만 rdd 든 dataframe 이든 immutable 이라서 수정할 수없다.

In [115]:
mpd  = means.toPandas() # pandas 로 데이터를 받아옴

In [116]:
type (mpd)

pandas.core.frame.DataFrame

In [118]:
mpd.to_dict('records') # 판다스 쉽게 , 키벨류 로 집어넣는다.  판다스에있는것을 딕셔너리 타입으로 바꾸기.

[{'id': 4.0,
  'weight': 140.28333333333333,
  'height': 5.471428571428572,
  'age': 40.4}]

In [119]:
mpd = mpd.to_dict('records')[0]

In [121]:
mpd # 키벨류만 남기기 . 리스트라서 0 인덱싱

{'id': 4.0,
 'weight': 140.28333333333333,
 'height': 5.471428571428572,
 'age': 40.4}

In [122]:
mpd['gender'] = 'X'

In [123]:
mpd

{'id': 4.0,
 'weight': 140.28333333333333,
 'height': 5.471428571428572,
 'age': 40.4,
 'gender': 'X'}


mpd 는 평균데이터 만들어놈

In [126]:
df = df.fillna(mpd)

In [127]:
df.show()

+---+------------------+------+---+------+
| id|            weight|height|age|gender|
+---+------------------+------+---+------+
|  1|             143.5|   5.6| 28|     M|
|  2|             167.2|   5.4| 45|     M|
|  3|140.28333333333333|   5.2| 40|     X|
|  4|             144.5|   5.9| 33|     M|
|  5|             133.2|   5.7| 54|     F|
|  6|             124.1|   5.2| 40|     F|
|  7|             129.2|   5.3| 42|     M|
+---+------------------+------+---+------+



In [128]:
# NULL 에 평균값이 대체됨.

In [129]:
#미싱데이터가 많은 컬럼은 날려버림.

 뉴메리컬 데이터가 있을때, 
# outlier 라고 해서 정규분포로 봤을때 비정상적인   데이터들을 어떻게 할것인가 . 

# 통계적인 방법에 의해서 날리는것 을 알아보자

In [132]:
df = spark.createDataFrame(
    [
        (1, 143.5, 5.3, 28, "M"),
        (2, 154.2, 5.5, 45, "M"),
        (3, 324.3, 5.1, 99, "F"),
        (4, 144.5, 5.5, 33, "M"),
        (5, 133.2, 5.4, 54, "F"),
        (6, 124.1, 5.1, 21, "F"),
        (7, 129.2, 5.3, 42, "M"),    
    ],
      ["id", "weight", "height", "age", "gender"]
)

In [133]:
quantiles = df.approxQuantile("weight",[0.25,0.75],0.05)

In [134]:
quantiles # 로어 바운드  , 어퍼바운드 가 생김

[129.2, 154.2]

In [135]:
IQR = quantiles[1] - quantiles[0] # 이거는 중간값이됨. 그러고나서 바운드를 만들어준다.

In [136]:
bounds = [quantiles[0]- 1.5*IQR,quantiles[1]+1.5*IQR]

In [138]:
bounds # weight 에 대해서 통계적으로 

[91.69999999999999, 191.7]

In [150]:
col = ["weight", "height", "age"]
bounds = {}

for c in col:
    quantiels = df.approxQuantile(c, [0.25, 0.75], 0.05)
    IQR = quantiels[1] - quantiels[0]
    bounds[c] = [quantiels[0]-1.5*IQR, quantiels[1]+1.5*IQR]

In [ ]:
# 컬럼에대해서 제일 높은범위랑 낮은범위랑. 만든다


In [152]:
#원래 데이터에 select를 할건데 , weight 가 우리가만든 바운드에 낮은값 보다 작은지.
df.select(df["weight"] < bounds["weight"][0]).show()
df.select(df["weight"] > bounds["weight"][1]).show()

+----------------------------+
|(weight < 91.69999999999999)|
+----------------------------+
|                       false|
|                       false|
|                       false|
|                       false|
|                       false|
|                       false|
|                       false|
+----------------------------+

+----------------+
|(weight > 191.7)|
+----------------+
|           false|
|           false|
|            true|
|           false|
|           false|
|           false|
|           false|
+----------------+



In [154]:
outlier =  df.select(
    *["id"] + [((df[c] < bounds[c][0]) |
    (df[c] > bounds[c][1])).alias(c+"_O") for c in col]
)

In [155]:
joinTable = df.join(outlier, on="id")

In [156]:
joinTable.filter("!weight_O").select("id", "weight", "height", "age").show()

+---+------+------+---+
| id|weight|height|age|
+---+------+------+---+
|  7| 129.2|   5.3| 42|
|  6| 124.1|   5.1| 21|
|  5| 133.2|   5.4| 54|
|  1| 143.5|   5.3| 28|
|  2| 154.2|   5.5| 45|
|  4| 144.5|   5.5| 33|
+---+------+------+---+



In [146]:
# 느낌표를 써서 !outlier 들을 뺀 값을 가져옴


In [147]:
#오류가 났지만 다시 보기

In [148]:
#데이터 수집 , 스토리지넣어서 크리닝 - > 머신러닝 